In [2]:
import sys
sys.path.append('../')

In [3]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.metrics.functional import accuracy
import torchdiffeq

from torchdyn.models import *; from torchdyn import *

# Image Classification

In this notebook we explore standard image classification on MNIST and CIFAR10 with convolutional neural ODE variants.
* Depth-invariant neural ODE
* Galerkin neural ODE (GalNODE)

In the following notebooks we'll explore `augmentation` strategies that can be easily applied to the models below with the flexible `torchdyn` API. Here, we use simple `0-augmentation` (the ANODE model).

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
batch_size=256
size=28
path_to_data='../data/mnist_data'

all_transforms = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

train_data = datasets.MNIST(path_to_data, train=True, download=True,
                            transform=all_transforms)
test_data = datasets.MNIST(path_to_data, train=False,
                           transform=all_transforms)

trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

The **Learner** is then defined as:

In [7]:
class Learner(pl.LightningModule):
    def __init__(self, model:nn.Module):
        super().__init__()
        self.lr = 1e-3
        self.model = model
        self.iters = 0.
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        self.iters += 1.
        x, y = batch   
        x, y = x.to(device), y.to(device)
        y_hat = self.model(x)   
        loss = nn.CrossEntropyLoss()(y_hat, y)
        print(f'\r{self.lr}', end='')
        epoch_progress = self.iters / self.loader_len
        acc = accuracy(y_hat, y)
        nfe = model[1].nfe ; model[1].nfe = 0
        tqdm_dict = {'train_loss': loss, 'accuracy': acc, 'NFE': nfe}
        logs = {'train_loss': loss, 'epoch': epoch_progress}
        return {'loss': loss, 'progress_bar': tqdm_dict, 'log': logs}   

    def test_step(self, batch, batch_nb):
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = self(x)
        acc = accuracy(y_hat, y)
        return {'test_loss': nn.CrossEntropyLoss()(y_hat, y), 'test_accuracy': acc}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_accuracy'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'avg_test_accuracy': avg_acc,
                'log': logs, 'progress_bar': logs}
    
    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=5e-5)
        sched = {'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(opt),
                 'monitor': 'loss', 
                 'interval': 'step',
                 'frequency': 10  }
        return [opt], [sched]

    def train_dataloader(self):
        self.loader_len = len(trainloader)
        return trainloader

    def test_dataloader(self):
        self.test_loader_len = len(trainloader)
        return testloader

## Depth-Invariant Conv Neural ODE 

In [8]:
func = nn.Sequential(nn.Conv2d(11, 12, 3, padding=1),
                     nn.Softplus(),
                     nn.Conv2d(12, 12, 5, padding=2),
                     nn.Softplus(),
                     nn.Conv2d(12, 11, 3, padding=1),                   
                     ).to(device)

neuralDE = NeuralDE(func, 
                   solver='rk4',
                   sensitivity='autograd',
                   s_span=torch.linspace(0, 1, 5)).to(device)

model = nn.Sequential(Augmenter(augment_dims=10),
                      neuralDE,
                      nn.Conv2d(11, 1, 3, padding=1),
                      nn.Flatten(),                     
                      nn.Linear(28*28, 10)).to(device)


In [9]:
learn = Learner(model)
trainer = pl.Trainer(max_epochs=10,
                     gpus=1,
                     progress_bar_refresh_rate=1,
                     )

trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 13 K  
/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


0.001


/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

2 epochs are not enough. Feel free to keep training and using all kinds of scheduling and optimization tricks :)

In [31]:
trainer.test(learn)

--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_accuracy': tensor(0.9187, device='cuda:0'),
 'avg_test_loss': tensor(0.2766, device='cuda:0'),
 'test_loss': tensor(0.2766, device='cuda:0')}
--------------------------------------------------------------------------------



## Galerkin Data-Controlled Conv Neural ODE (IL-Augmentation)

In [10]:
func = nn.Sequential(DataControl(),
                     DepthCat(1),
                     GalConv2d(10+10, 12, 3, padding=1, expfunc=FourierExpansion, n_harmonics=5),
                     nn.Softplus(),
                     DataControl(),
                     DepthCat(1),
                     GalConv2d(22, 10, 3, padding=1, expfunc=FourierExpansion, n_harmonics=5),
                     nn.Tanh()
                     )

neuralDE = NeuralDE(func, 
                   solver='rk4',
                   sensitivity='autograd',
                   s_span=torch.linspace(0, 1, 5)).to(device)

model = nn.Sequential(Augmenter(augment_idx=1, augment_func=nn.Conv2d(1, 9, 3, padding=1)),
                      neuralDE,
                      nn.Conv2d(10, 1, 3, padding=1),
                      nn.Flatten(),                     
                      nn.Linear(28*28, 10)).to(device)


In [11]:
learn = Learner(model)
trainer = pl.Trainer(max_epochs=10,
                     gpus=1,
                     progress_bar_refresh_rate=1,
                     )

trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 49 K  


0.001


1

In [13]:
trainer.test(learn)

/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_accuracy': tensor(0.9580, device='cuda:0'),
 'avg_test_loss': tensor(0.1408, device='cuda:0'),
 'test_loss': tensor(0.1408, device='cuda:0')}
--------------------------------------------------------------------------------

